In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import json
from sklearn.model_selection import train_test_split
from gensim.models import word2vec
from sklearn.externals import joblib
from google.colab import files
import pickle
from sklearn.linear_model import LogisticRegression

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
def start():
    data_file = open("/content/drive/My Drive/Проект 'рекомендательная система для магазина'/data.json",'r')
    emb_file = open("/content/drive/My Drive/Проект 'рекомендательная система для магазина'/word2vec.pkl",'rb')
    ml_file = open("/content/drive/My Drive/Проект 'рекомендательная система для магазина'/clf.pkl",'rb')
    dictionary = json.load(data_file)
    model = pickle.load(emb_file)
    w2v = dict(zip(model.wv.index2word, model.wv.vectors))
    clf = pickle.load(ml_file)
    data_file.close()
    emb_file.close()
    ml_file.close()
    return dictionary, w2v, clf

def end():
    data_file = open("/content/drive/My Drive/Проект 'рекомендательная система для магазина'/model.json",'w')
    json.dump(dictionary, data_file)

def retraining():
    one_buyers=[]
    last_thing=[]
    for i in dictionary.keys():
        if len(dictionary[i])==1:
            one_buyers.append(i)
    for i in one_buyers:
        del dictionary[i]
    for i in dictionary.keys():
        last_thing.append(dictionary[i][-1])
    data=[]
    for i in dictionary.keys():
        data.append(dictionary[i])

    model = word2vec.Word2Vec(data, size=300, window=3, workers=4)
    w2v = dict(zip(model.wv.index2word, model.wv.vectors))
    data_mean=mean_vectorizer(w2v).fit(data).transform(data)

    clf = LogisticRegression(C=1, random_state=241,multi_class='ovr', tol=0.001)
    clf.fit(data_mean, last_thing)

    emb_file = open("/content/drive/My Drive/Проект 'рекомендательная система для магазина'/word2vec.pkl",'wb')
    ml_file = open("/content/drive/My Drive/Проект 'рекомендательная система для магазина'/clf.pkl",'wb')
    pickle.dump(model, emb_file)
    pickle.dump(clf, ml_file)
    emb_file.close()
    ml_file.close()
    return w2v, clf

def prediction(ID_prod, ID_cust):
    dictionary[ID_cust].append(ID_prod)
    data = dictionary[ID_cust]
    data = np.expand_dims(data, axis=0)
    data_mean=mean_vectorizer(w2v).fit(data).transform(data)
    proba = clf.predict_proba(data_mean)
    index = np.argpartition(proba[0], -5)[-5:]
    classes = clf.classes_[index]
    return classes

In [ ]:
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [ ]:
dictionary, w2v, clf = start() # 3 штуки, которые будут в памяти всегда лежать

In [ ]:
pred = prediction('85048', "13085.0") #первый аргумент - ID товара, второй - ID покупателя
print(pred) # 5 наиболее вероятных товаров

['85123A' '21232' '22138' 'M' 'POST']


In [ ]:
w2v, clf = retraining()

In [ ]:
end()